In [2]:
def PhotovoltaicTotalPowerPredict(year, month):
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from statsmodels.tsa.api import Holt
    import itertools
    import warnings
    warnings.filterwarnings("ignore")


    data = pd.read_excel(r"D:\BONC\jobContent\PhotovoltaicPowerPrediction\DataCenterExport2019-10-09.xls")
    data['Timestamp'] = pd.to_datetime(data['日期'], format='%Y/%m/%d %H:%M:%S')
    data.index = data['Timestamp']
    resamplingData = data.resample("15T").mean()

    for AIndex in range(len(resamplingData)):
        if np.isnan(resamplingData.iloc[AIndex]["参数值"]):
            for BIndex in range(AIndex + 1, len(resamplingData)):
                if not np.isnan(resamplingData.iloc[BIndex]["参数值"]):
                    resamplingData.iloc[AIndex]["参数值"] = (resamplingData.iloc[AIndex - 1]["参数值"] + resamplingData.iloc[BIndex]["参数值"]) / 2
                    break
    
    SmoothingLevelList = np.arange(0.01, 1, 0.01)
    SmoothingSlopeList = np.arange(0.01, 1, 0.01)
    ErrorValue = 0.1
    StandardErrorValue = 0.02
    messageList = []
   
    train = resamplingData[96 * TrainStartNumber : 96 * TrainEndNumber]
    test = resamplingData[96 * TestStartNumber : 96 * TestEndNumber]
    y_hat_avg = test.copy()
    
    parametersList = list(itertools.product(SmoothingLevelList, SmoothingSlopeList))
    
    for everyParameter in parametersList:
        try:
            fit = Holt(np.asarray(train['参数值'])).fit(smoothing_level=everyParameter[0], smoothing_slope=everyParameter[1])
            y_hat_avg['Holt_linear'] = fit.forecast(len(test))
        except:
            continue

        predictValue = y_hat_avg['Holt_linear'][-1] - y_hat_avg['Holt_linear'][0]
        realValue = resamplingData.iloc[96 * TestEndNumber - 1]["参数值"] - resamplingData.iloc[96 * TestStartNumber]["参数值"]
        errorValue = abs((predictValue - realValue) / realValue)
        messageList.append([realValue, predictValue, errorValue, [everyParameter[0], everyParameter[1]]])


        if errorValue < StandardErrorValue:
            BestSmoothingLevel = everyParameter[0]
            BestSmoothingSlope = everyParameter[1]
            bestParameter = [everyParameter[0], everyParameter[1]]

            print("真实值：{}，预测值：{}， 最小误差：{}，对应参数：{}".format(realValue, predictValue, errorValue, 
                                                  [everyParameter[0], everyParameter[1]]))
            
            if ErrorValue < StandardErrorValue:
                break